In [ ]:
import sqlite3

# Create a connection to the database
conn = sqlite3.connect('marketing_research9.db')

# Create a cursor object
cur = conn.cursor()

# Create the employees table
cur.execute('''
CREATE TABLE campaigns (
    CampaignId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    CampaignName NVARCHAR(50) NOT NULL,
    StartDate DATETIME NOT NULL,
    EndDate DATETIME NOT NULL,
    Budget REAL NOT NULL
)
''')

cur.execute('''
CREATE TABLE customers (
    CustomerId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    FirstName NVARCHAR(20) NOT NULL,
    LastName NVARCHAR(20) NOT NULL,
    Email NVARCHAR(60) NOT NULL,
    Phone NVARCHAR(20),
    Address NVARCHAR(70)
)''')

cur.execute('''
CREATE TABLE customer_responses (
    ResponseId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    CampaignId INTEGER NOT NULL,
    CustomerId INTEGER NOT NULL,
    ResponseDate DATETIME NOT NULL,
    Response NVARCHAR(200),
    FOREIGN KEY (CampaignId) REFERENCES campaigns (CampaignId),
    FOREIGN KEY (CustomerId) REFERENCES customers (CustomerId)
)''')

cur.execute('''
CREATE TABLE market_segments (
    SegmentId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    SegmentName NVARCHAR(30) NOT NULL,
    Description NVARCHAR(100)
)''')

cur.execute('''
CREATE TABLE customer_segments (
    CustomerId INTEGER NOT NULL,
    SegmentId INTEGER NOT NULL,
    PRIMARY KEY (CustomerId, SegmentId),
    FOREIGN KEY (CustomerId) REFERENCES customers (CustomerId),
    FOREIGN KEY (SegmentId) REFERENCES market_segments (SegmentId)
)''')

cur.execute('''
CREATE TABLE products (
    ProductId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    ProductName NVARCHAR(50) NOT NULL,
    Description NVARCHAR(100),
    Price REAL NOT NULL
)''')

cur.execute('''
CREATE TABLE product_responses (
    ResponseId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    ProductId INTEGER NOT NULL,
    CustomerId INTEGER NOT NULL,
    ResponseDate DATETIME NOT NULL,
    Response NVARCHAR(200),
    FOREIGN KEY (ProductId) REFERENCES products (ProductId),
    FOREIGN KEY (CustomerId) REFERENCES customers (CustomerId)
)''')

cur.execute('''
CREATE TABLE advertising_channels (
    ChannelId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    ChannelName NVARCHAR(30) NOT NULL,
    Description NVARCHAR(100)
)''')

cur.execute('''
CREATE TABLE campaign_channels (
    CampaignId INTEGER NOT NULL,
    ChannelId INTEGER NOT NULL,
    PRIMARY KEY (CampaignId, ChannelId),
    FOREIGN KEY (CampaignId) REFERENCES campaigns (CampaignId),
    FOREIGN KEY (ChannelId) REFERENCES advertising_channels (ChannelId)
)''')

cur.execute('''
CREATE TABLE sales_data (
    SaleId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    ProductId INTEGER NOT NULL,
    CustomerId INTEGER NOT NULL,
    SaleDate DATETIME NOT NULL,
    Quantity INTEGER NOT NULL,
    Revenue REAL NOT NULL,
    FOREIGN KEY (ProductId) REFERENCES products (ProductId),
    FOREIGN KEY (CustomerId) REFERENCES customers (CustomerId)
)''')

cur.execute('''
CREATE TABLE distribution_channels (
    ChannelId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    ChannelName NVARCHAR(30) NOT NULL,
    Description NVARCHAR(100)
)''')

cur.execute('''
CREATE TABLE product_distribution (
    ProductId INTEGER NOT NULL,
    ChannelId INTEGER NOT NULL,
    PRIMARY KEY (ProductId, ChannelId),
    FOREIGN KEY (ProductId) REFERENCES products (ProductId),
    FOREIGN KEY (ChannelId) REFERENCES distribution_channels (ChannelId)
)''')

cur.execute('''
CREATE TABLE public_relations (
    PRId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    PRType NVARCHAR(20) NOT NULL,
    Description NVARCHAR(100),
    Date DATETIME NOT NULL
)''')

cur.execute('''
CREATE TABLE pr_campaigns (
    PRId INTEGER NOT NULL,
    CampaignId INTEGER NOT NULL,
    PRIMARY KEY (PRId, CampaignId),
    FOREIGN KEY (PRId) REFERENCES public_relations (PRId),
    FOREIGN KEY (CampaignId) REFERENCES campaigns (CampaignId)
)''')
# Commit the changes
conn.commit()

# Close the connection
conn.close()

In [ ]:
import csv
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker to generate synthetic data
fake = Faker()

# Define the unique columns across all tables
unique_columns = [
    "CampaignId", "CampaignName", "StartDate", "EndDate", "Budget",
    "CustomerId", "FirstName", "LastName", "Email", "Phone", "Address",
    "ResponseId", "ResponseDate", "Response",
    "SegmentId", "SegmentName", "Description",
    "ProductId", "ProductName", "Price",
    "ChannelId", "ChannelName",
    "SaleId", "SaleDate", "Quantity", "Revenue",
    "PRId", "PRType", "PRDate", "PRDescription"
]

# Number of synthetic rows to generate
num_rows = 500

# Initialize counters for auto-incrementing ID fields
campaign_id = 1
customer_id = 1
response_id = 1
segment_id = 1
product_id = 1
channel_id = 1
sale_id = 1
pr_id = 1

# Function to generate synthetic data for each column
def generate_synthetic_data():
    global campaign_id, customer_id, response_id, segment_id, product_id, channel_id, sale_id, pr_id

    data = {
        "CampaignId": campaign_id,
        "CampaignName": f"CampaignName{campaign_id}",
        "StartDate": fake.date_between(start_date="-2y", end_date="today"),
        "EndDate": fake.date_between(start_date="today", end_date="+2y"),
        "Budget": round(fake.random_number(digits=5), 2),
        "CustomerId": customer_id,
        "FirstName": fake.first_name(),
        "LastName": fake.last_name(),
        "Email": fake.email(),
        "Phone": fake.phone_number(),
        "Address": fake.address(),
        "ResponseId": response_id,
        "ResponseDate": fake.date_time_between(start_date="-2y", end_date="now"),
        "Response": fake.sentence(),
        "SegmentId": segment_id,
        "SegmentName": f"segment{segment_id}",
        "Description": fake.text(max_nb_chars=100),
        "ProductId": product_id,
        "ProductName": f"product{product_id}",
        "Price": round(fake.random_number(digits=3), 2),
        "ChannelId": channel_id,
        "ChannelName": f"Channel{channel_id}",
        "SaleId": sale_id,
        "SaleDate": fake.date_time_between(start_date="-2y", end_date="now"),
        "Quantity": fake.random_int(min=1, max=100),
        "Revenue": round(fake.random_number(digits=4), 2),
        "PRId": pr_id,
        "PRType": f"pr_type{pr_id}",
        "PRDate": fake.date_time_between(start_date="-2y", end_date="now"),
        "PRDescription": f"pr_desc{pr_id}"
    }

    # Increment ID values for the next record
    campaign_id += 1
    customer_id += 1
    response_id += 1
    segment_id += 1
    product_id += 1
    channel_id += 1
    sale_id += 1
    pr_id += 1
    #pr_desc +=1
    #pr_type +=1

    return data

# Generate synthetic data and write to CSV
with open("synthetic_data.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=unique_columns)
    writer.writeheader()
    for _ in range(num_rows):
        writer.writerow(generate_synthetic_data())

print("Synthetic data CSV file 'synthetic_data.csv' created.")


Synthetic data CSV file 'synthetic_data.csv' created.


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00


In [ ]:
product_names = [
"HubSpot",
"Marketo",
"Pardot",
"Salesforce",
"Mailchimp",
"Hootsuite",
"Buffer",
"Sprout Social",
"Adobe Campaign",
"Google Analytics",
"Facebook Ads Manager",
"Twitter Ads",
"LinkedIn Ads",
"Instagram Ads",
"YouTube Ads",
"Google Ads",
"Bing Ads",
"SEMrush",
"Ahrefs",
"Moz",
"BuzzStream",
"Cision",
"Meltwater",
"Brand24",
"Sprinklr",
"Oracle Marketing Cloud",
"SAP Marketing Cloud",
"IBM Watson Campaign Automation",
"Microsoft Dynamics 365",
"Zoho Marketing Automation",
"Infusionsoft",
"ActiveCampaign",
"Constant Contact",
"GetResponse",
"Klaviyo",
"Iterable",
"Braze",
"Leanplum",
"Swrve",
"Urban Airship",
"Localytics",
"Mixpanel",
"Kissmetrics",
"Crazy Egg",
"Hotjar",
"Unbounce",
"Instapage",
"Wix",
"Squarespace",
"Weebly",
"WordPress",
"Shopify",
"BigCommerce",
"Magento",
"OpenCart",
"Salesforce Commerce Cloud",
"Demandware",
"SAP Commerce Cloud",
"Oracle Commerce Cloud",
"IBM WebSphere Commerce",
"Microsoft Dynamics 365 Commerce",
"Influitive",
"Advocacy Platform",
"Bazaarvoice",
"PowerReviews",
"Yotpo",
"Trustpilot",
"ReviewTrackers",
"BirdEye",
"AskNicely",
"Delighted",
"SurveyMonkey",
"Qualtrics",
"Medallia",
"InMoment",
"AskForTask",
"UserTesting",
"TryMyUI",
"Userlytics",
"What Users Do",
"UserZoom",
"AnswerLab",
"UserInput",
"Validately",
"UserFeel"
]
import random




segment_names = [
    "Marketing Automation",
    "Social Media Management",
    "Analytics",
    "Advertising",
    "Email Marketing",
    "Customer Experience",
    "Content Management",
    "E-commerce",
    "SEO",
    "Customer Service",
    "Sales",
    "IT",
    "Finance",
    "HR",
    "Operations"
]

# Create a dictionary to map segment names to IDs
segments = {name: f"SEG{i+1}" for i, name in enumerate(segment_names)}

# Randomly assign products to segments
product_segments = {}
for product in random.sample(product_names, 30):
    segment_id = random.choice(list(segments.values()))
    product_segments[product] = segment_id

# Print the product IDs, names, and segments
for i, (product, segment_id) in enumerate(product_segments.items()):
    product_id = f"PROD{i+1}"
    segment_name = [name for name, sid in segments.items() if sid == segment_id][0]
    print(f"Product ID: {product_id}, Product Name: {product}, Segment ID: {segment_id}, Segment Name: {segment_name}")


Product ID: PROD1, Product Name: AskForTask, Segment ID: SEG8, Segment Name: E-commerce
Product ID: PROD2, Product Name: SAP Marketing Cloud, Segment ID: SEG11, Segment Name: Sales
Product ID: PROD3, Product Name: Buffer, Segment ID: SEG12, Segment Name: IT
Product ID: PROD4, Product Name: TryMyUI, Segment ID: SEG3, Segment Name: Analytics
Product ID: PROD5, Product Name: Salesforce Commerce Cloud, Segment ID: SEG5, Segment Name: Email Marketing
Product ID: PROD6, Product Name: Wix, Segment ID: SEG14, Segment Name: HR
Product ID: PROD7, Product Name: PowerReviews, Segment ID: SEG7, Segment Name: Content Management
Product ID: PROD8, Product Name: Twitter Ads, Segment ID: SEG13, Segment Name: Finance
Product ID: PROD9, Product Name: Qualtrics, Segment ID: SEG13, Segment Name: Finance
Product ID: PROD10, Product Name: IBM WebSphere Commerce, Segment ID: SEG4, Segment Name: Advertising
Product ID: PROD11, Product Name: Swrve, Segment ID: SEG1, Segment Name: Marketing Automation
Product ID

In [10]:
import pandas as pd
import random

# Define the 100 product names
product_names = product_names = [
"HubSpot",
"Marketo",
"Pardot",
"Salesforce",
"Mailchimp",
"Hootsuite",
"Buffer",
"Sprout Social",
"Adobe Campaign",
"Google Analytics",
"Facebook Ads Manager",
"Twitter Ads",
"LinkedIn Ads",
"Instagram Ads",
"YouTube Ads",
"Google Ads",
"Bing Ads",
"SEMrush",
"Ahrefs",
"Moz",
"BuzzStream",
"Cision",
"Meltwater",
"Brand24",
"Sprinklr",
"Oracle Marketing Cloud",
"SAP Marketing Cloud",
"IBM Watson Campaign Automation",
"Microsoft Dynamics 365",
"Zoho Marketing Automation",
"Infusionsoft",
"ActiveCampaign",
"Constant Contact",
"GetResponse",
"Klaviyo",
"Iterable",
"Braze",
"Leanplum",
"Swrve",
"Urban Airship",
"Localytics",
"Mixpanel",
"Kissmetrics",
"Crazy Egg",
"Hotjar",
"Unbounce",
"Instapage",
"Wix",
"Squarespace",
"Weebly",
"WordPress",
"Shopify",
"BigCommerce",
"Magento",
"OpenCart",
"Salesforce Commerce Cloud",
"Demandware",
"SAP Commerce Cloud",
"Oracle Commerce Cloud",
"IBM WebSphere Commerce",
"Microsoft Dynamics 365 Commerce",
"Influitive",
"Advocacy Platform",
"Bazaarvoice",
"PowerReviews",
"Yotpo",
"Trustpilot",
"ReviewTrackers",
"BirdEye",
"AskNicely",
"Delighted",
"SurveyMonkey",
"Qualtrics",
"Medallia",
"InMoment",
"AskForTask",
"UserTesting",
"TryMyUI",
"Userlytics",
"What Users Do",
"UserZoom",
"AnswerLab",
"UserInput",
"Validately",
"UserFeel"]
  # insert the 100 product names here

# Define the 15 segment names
segment_names = [
    "Marketing Automation",
    "Social Media Management",
    "Analytics",
    "Advertising",
    "Email Marketing",
    "Customer Experience",
    "Content Management",
    "E-commerce",
    "SEO",
    "Customer Service",
    "Sales",
    "IT",
    "Finance",
    "HR",
    "Operations"
]

# Create a dictionary to map segment names to IDs
segments = {name: f"SEG{i+1}" for i, name in enumerate(segment_names)}

# Randomly assign products to segments
product_segments = {}
for product in random.sample(product_names, 30):
    segment_id = random.choice(list(segments.values()))
    product_segments[product] = segment_id

# Create a DataFrame to store the results
df = pd.DataFrame(columns=["ProductId", "ProductName", "SegmentId", "SegmentName"])

# Populate the DataFrame
for i, (product, segment_id) in enumerate(product_segments.items()):
    product_id = f"PROD{i+1}"
    segment_name = [name for name, sid in segments.items() if sid == segment_id][0]
    df = pd.concat([df, pd.DataFrame([{"ProductId": product_id, "ProductName": product, "SegmentId": segment_id, "SegmentName": segment_name}])], ignore_index=True)

# Print the DataFrame
print(df)

   ProductId                ProductName SegmentId              SegmentName
0      PROD1                    Brand24     SEG15               Operations
1      PROD2           Constant Contact      SEG5          Email Marketing
2      PROD3                   Sprinklr     SEG12                       IT
3      PROD4                 Demandware      SEG1     Marketing Automation
4      PROD5                Squarespace      SEG4              Advertising
5      PROD6                   Unbounce     SEG12                       IT
6      PROD7                 Influitive      SEG9                      SEO
7      PROD8        SAP Marketing Cloud     SEG11                    Sales
8      PROD9     Microsoft Dynamics 365      SEG5          Email Marketing
9     PROD10  Salesforce Commerce Cloud      SEG9                      SEO
10    PROD11               Infusionsoft     SEG10         Customer Service
11    PROD12                     Weebly      SEG6      Customer Experience
12    PROD13             

In [11]:
!pip install faker

In [43]:
import csv
from faker import Faker
from datetime import datetime, timedelta
import pandas as pd
import random

# Initialize Faker to generate synthetic data
fake = Faker()

# Load the dataframe
#df = pd.read_csv('your_dataframe.csv')

# Define the unique columns across all tables
unique_columns = [
    "CampaignId", "CampaignName", "StartDate", "EndDate", "Budget",
    "CustomerId", "FirstName", "LastName", "Email", "Phone", "Address",
    "ResponseId", "ResponseDate", "Response",
    "ProductId", "ProductName", "Price",
    "SegmentId", "SegmentName",
    "ChannelId", "ChannelName",
    "SaleId", "SaleDate", "Quantity", "Revenue",
    "PRId", "PRType", "PRDate", "PRDescription"
]

# Number of synthetic rows to generate
num_rows = 500

# Initialize counters for auto-incrementing ID fields
campaign_id = 1
customer_id = 1
response_id = 1
channel_id = 1
sale_id = 1
pr_id = 1

# Function to generate synthetic data for each column
def generate_synthetic_data():
    global campaign_id, customer_id, response_id, channel_id, sale_id, pr_id

    # Randomly select a product and segment from the dataframe
    product = df.sample(1).iloc[0]
    product_id = product['ProductId']
    product_name = product['ProductName']
    segment_id = product['SegmentId']
    segment_name = product['SegmentName']
    ChannelName=random.choice([f"Channel_{i}" for i in range(1,10)])
    channel_id=int(ChannelName[::-1][0])
    campaign_id=random.choice([i for i in range(1,10)])
    pr_id=random.choice([i for i in range(1,20)])
    #customer_id=random.choice([i for i in range(1,30)])
    response_id=random.choice([i for i in range(1,50)])
    sale_id=random.choice([i for i in range(1,10)])
    data = {
        "CampaignId": campaign_id,
        "CampaignName": f"Campaign{campaign_id}",
        "StartDate": fake.date_between(start_date="-2y", end_date="today"),
        "EndDate": fake.date_between(start_date="today", end_date="+2y"),
        "Budget": round(fake.random_number(digits=5), 2),
        "CustomerId": customer_id,
        "FirstName": fake.first_name(),
        "LastName": fake.last_name(),
        "Email": fake.email(),
        "Phone": fake.phone_number(),
        "Address": fake.address(),
        "ResponseId": response_id,
        "ResponseDate": fake.date_time_between(start_date="-2y", end_date="now"),
        "Response": random.choice(['positive', 'negative', 'neutral']),
        "ProductId": product_id,
        "ProductName": product_name,
        "Price": round(fake.random_number(digits=3), 2),
        "SegmentId": segment_id,
        "SegmentName": segment_name,
        "ChannelId": channel_id,
        "ChannelName":ChannelName ,
        "SaleId": sale_id,
        "SaleDate": fake.date_time_between(start_date="-2y", end_date="now"),
        "Quantity": fake.random_int(min=1, max=100),
        "Revenue": round(fake.random_number(digits=4), 2),
        "PRId": pr_id,
        "PRType": f"pr_type{pr_id}",
        "PRDate": fake.date_time_between(start_date="-2y", end_date="now"),
        "PRDescription": f"pr_desc{pr_id}"
    }

    # Increment ID values for the next record
    #campaign_id += 1
    customer_id += 1
    #response_id += 1
    #channel_id += 1
    #sale_id += 1
    #pr_id += 1

    return data

# Generate synthetic data and write to CSV
with open("synthetic_data.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=unique_columns)
    writer.writeheader()
    for _ in range(num_rows):
        writer.writerow(generate_synthetic_data())

print("Synthetic data CSV file 'synthetic_data.csv' created.")

Synthetic data CSV file 'synthetic_data.csv' created.


In [44]:
pd.read_csv("synthetic_data.csv").groupby(["ProductId","ProductName","SegmentId","SegmentName"]).count()

,,,,CampaignId,CampaignName,StartDate,EndDate,Budget,CustomerId,FirstName,LastName,Email,Phone,...,ChannelId,ChannelName,SaleId,SaleDate,Quantity,Revenue,PRId,PRType,PRDate,PRDescription
ProductId,ProductName,SegmentId,SegmentName,,,,,,,,,,,,,,,,,,,,,
PROD1,Brand24,SEG15,Operations,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
PROD10,Salesforce Commerce Cloud,SEG9,SEO,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
PROD11,Infusionsoft,SEG10,Customer Service,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,22,22
PROD12,Weebly,SEG6,Customer Experience,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
PROD13,UserInput,SEG3,Analytics,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
PROD14,Instagram Ads,SEG9,SEO,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
PROD15,InMoment,SEG12,IT,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
PROD16,Salesforce,SEG5,Email Marketing,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
PROD17,Leanplum,SEG4,Advertising,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15


In [45]:
pd.read_csv("synthetic_data.csv").groupby(["ChannelId","ChannelName"]).count()

,,CampaignId,CampaignName,StartDate,EndDate,Budget,CustomerId,FirstName,LastName,Email,Phone,...,SegmentId,SegmentName,SaleId,SaleDate,Quantity,Revenue,PRId,PRType,PRDate,PRDescription
ChannelId,ChannelName,,,,,,,,,,,,,,,,,,,,,
1,Channel_1,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,53,53,53,53,53
2,Channel_2,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,58,58,58,58,58,58
3,Channel_3,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
4,Channel_4,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
5,Channel_5,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
6,Channel_6,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
7,Channel_7,57,57,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
8,Channel_8,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
9,Channel_9,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,58,58,58,58,58,58


In [46]:
pd.read_csv("synthetic_data.csv").groupby(["CampaignId","CampaignName"]).count()

,,StartDate,EndDate,Budget,CustomerId,FirstName,LastName,Email,Phone,Address,ResponseId,...,ChannelId,ChannelName,SaleId,SaleDate,Quantity,Revenue,PRId,PRType,PRDate,PRDescription
CampaignId,CampaignName,,,,,,,,,,,,,,,,,,,,,
1,Campaign1,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,58,58,58,58,58,58
2,Campaign2,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
3,Campaign3,51,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51
4,Campaign4,60,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
5,Campaign5,50,50,50,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
6,Campaign6,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,53,53,53,53,53
7,Campaign7,54,54,54,54,54,54,54,54,54,54,...,54,54,54,54,54,54,54,54,54,54
8,Campaign8,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
9,Campaign9,65,65,65,65,65,65,65,65,65,65,...,65,65,65,65,65,65,65,65,65,65


In [48]:
pd.read_csv("synthetic_data.csv").groupby(["CustomerId","FirstName"]).count()

,,CampaignId,CampaignName,StartDate,EndDate,Budget,LastName,Email,Phone,Address,ResponseId,...,ChannelId,ChannelName,SaleId,SaleDate,Quantity,Revenue,PRId,PRType,PRDate,PRDescription
CustomerId,FirstName,,,,,,,,,,,,,,,,,,,,,
1,Brett,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,Heather,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,Brian,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Mitchell,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,Rachel,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,Megan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
497,Brittany,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
498,John,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [49]:
pd.read_csv("synthetic_data.csv").groupby(["PRId","PRType"]).count()

,,CampaignId,CampaignName,StartDate,EndDate,Budget,CustomerId,FirstName,LastName,Email,Phone,...,SegmentId,SegmentName,ChannelId,ChannelName,SaleId,SaleDate,Quantity,Revenue,PRDate,PRDescription
PRId,PRType,,,,,,,,,,,,,,,,,,,,,
1,pr_type1,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
2,pr_type2,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
3,pr_type3,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
4,pr_type4,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
5,pr_type5,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29
6,pr_type6,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
7,pr_type7,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
8,pr_type8,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
9,pr_type9,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
